## Frequent Itemset and Association Rule Mining

Consider the text data from MMDS as an example:

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import combinations

spark = SparkSession \
        .builder \
        .appName("frequent_itemsets") \
        .getOrCreate()

data = [(0, ['cat', 'and', 'dog', 'bits']),
        (1, ['yahoo', 'news', 'claims', 'a', 'cat', 'mated', 'with', 'dog', 'and', 'produced', 'viable', 'offspring']),
        (2, ['cat', 'killer', 'likely', 'is', 'a', 'big', 'dog']),
        (3, ['professional', 'free', 'advice', 'on', 'dog', 'training', 'puppy']),
        (4, ['cat', 'and', 'kitten', 'training', 'behavior']),
        (5, ['dog', '&', 'cat', 'provides', 'training', 'in', 'eugene', 'oregon']),
        (6, ['dog', 'and', 'cat', 'is', 'a', 'slang', 'term', 'used', 'by', 'police', 'officers', 'for', 'male-female', 'relationship']),
        (7, ['shop', 'for', 'your', 'show', 'dog', 'grooming', 'and', 'pet', 'supplies'])
]

df = spark.createDataFrame(data, ["id", "basket"])

24/10/09 23:17:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## A Priori Algorithm

How can we identify frequent itemsets and associations in this dataset? While the A Priori Algorithm discussed in MMDS is not implemented in Spark, we can write our version. Let's walk through the different steps of the algorithm (as well as how they can be implemented in Spark) and then we can consolidate everything into a single function:

In [4]:
s = 3
# Find frequent singletons
L1 = df.rdd.flatMap(lambda x: [((i,), 1) for i in x.basket]) \
           .reduceByKey(lambda x, y: x + y) \
           .filter(lambda x: x[1] >= s) \
           .collect()
Lk_1 = dict(L1)
freq_itemsets = Lk_1
freq_itemsets

{('dog',): 7, ('a',): 3, ('training',): 3, ('cat',): 6, ('and',): 5}

In [5]:
C2 = df.rdd.flatMap(lambda x: [i for i in combinations(x.basket, 2)]) \
                   .map(lambda x: (x, set([tuple(sorted(i)) for i in combinations(x, 1)]))) \
                   .filter(lambda x: set(x[1]).issubset(Lk_1)) \
                   .map(lambda x: (tuple(sorted(x[0])), 1))
C2.collect()

[(('and', 'cat'), 1),
 (('cat', 'dog'), 1),
 (('and', 'dog'), 1),
 (('a', 'cat'), 1),
 (('a', 'dog'), 1),
 (('a', 'and'), 1),
 (('cat', 'dog'), 1),
 (('and', 'cat'), 1),
 (('and', 'dog'), 1),
 (('a', 'cat'), 1),
 (('cat', 'dog'), 1),
 (('a', 'dog'), 1),
 (('dog', 'training'), 1),
 (('and', 'cat'), 1),
 (('cat', 'training'), 1),
 (('and', 'training'), 1),
 (('cat', 'dog'), 1),
 (('dog', 'training'), 1),
 (('cat', 'training'), 1),
 (('and', 'dog'), 1),
 (('cat', 'dog'), 1),
 (('a', 'dog'), 1),
 (('and', 'cat'), 1),
 (('a', 'and'), 1),
 (('a', 'cat'), 1),
 (('and', 'dog'), 1)]

In [6]:
L2 = C2.reduceByKey(lambda x, y: x + y) \
       .filter(lambda x: x[1] >= s) \
       .collect()

L2 = dict(L2)
L2

{('a', 'dog'): 3,
 ('and', 'cat'): 4,
 ('and', 'dog'): 4,
 ('cat', 'dog'): 5,
 ('a', 'cat'): 3}

In [7]:
C3 = df.rdd.flatMap(lambda x: [i for i in combinations(x.basket, 3)]) \
           .map(lambda x: (x, set([tuple(sorted(i)) for i in combinations(x, 2)]))) \
           .filter(lambda x: set(x[1]).issubset(L2)) \
           .map(lambda x: (tuple(sorted(x[0])), 1)) \

L3 = C3.reduceByKey(lambda x, y: x + y) \
       .filter(lambda x: x[1] >= s) \
       .collect()
L3 = dict(L3)
L3

{('a', 'cat', 'dog'): 3, ('and', 'cat', 'dog'): 3}

Bringing it all together into a single function:

In [8]:
# Discuss: Critique this implementation
# in what scenarios might this fail to scale? What might you do to fix it?
def apriori(df, support=3):
    # Find frequent singletons
    L1 = df.rdd.flatMap(lambda x: [((i,), 1) for i in x.basket]) \
               .reduceByKey(lambda x, y: x + y) \
               .filter(lambda x: x[1] >= s) \
               .collect()
    Lk_1 = dict(L1)
    freq_itemsets = Lk_1

    # Find frequent doubletons and higher
    k = 2
    while Lk_1:
        Lk = df.rdd.flatMap(lambda x: [i for i in combinations(x.basket, k)]) \
                   .map(lambda x: (x, set([tuple(sorted(i)) for i in combinations(x, k - 1)]))) \
                   .filter(lambda x: set(x[1]).issubset(Lk_1)) \
                   .map(lambda x: (tuple(sorted(x[0])), 1)) \
                   .reduceByKey(lambda x, y: x + y) \
                   .filter(lambda x: x[1] >= s) \
                   .collect()
        Lk_1 = dict(Lk)
        freq_itemsets.update(Lk_1)
        k += 1

    return freq_itemsets

freq_itemsets = apriori(df)
freq_itemsets

{('dog',): 7,
 ('a',): 3,
 ('training',): 3,
 ('cat',): 6,
 ('and',): 5,
 ('a', 'dog'): 3,
 ('and', 'cat'): 4,
 ('and', 'dog'): 4,
 ('cat', 'dog'): 5,
 ('a', 'cat'): 3,
 ('a', 'cat', 'dog'): 3,
 ('and', 'cat', 'dog'): 3}

...and another function to generate association rules:

In [9]:
def generate_association_rules(freq_itemsets, total_count, confidence_threshold=0.8):
    # Serial computation of association rules + confidence/interest
    # based on 'freq_itemsets' dictionary and MMDS definitions
    rules = []
    for itemset, support in freq_itemsets.items():
        # Generate all non-empty subsets of the itemset
        subsets = [set(x) for i in range(1, len(itemset)) for x in combinations(itemset, i)]
        for subset in subsets:
            subset = tuple(subset)
            remaining = tuple(set(itemset) - set(subset))
            if remaining:
                # Calculate confidence
                subset_support = freq_itemsets.get(subset, 0)
                if subset_support > 0:
                    confidence = support / subset_support
                    if confidence >= confidence_threshold:
                        # Calculate interest
                        remaining_support = freq_itemsets.get(remaining, 0)
                        prob_remaining = remaining_support / total_count
                        interest = confidence - prob_remaining
                        rules.append((subset, remaining, confidence, interest))
    return rules

generate_association_rules(freq_itemsets, df.count())

[(('a',), ('dog',), 1.0, 0.125),
 (('and',), ('cat',), 0.8, 0.050000000000000044),
 (('and',), ('dog',), 0.8, -0.07499999999999996),
 (('cat',), ('dog',), 0.8333333333333334, -0.04166666666666663),
 (('a',), ('cat',), 1.0, 0.25),
 (('a',), ('dog', 'cat'), 1.0, 1.0),
 (('a', 'cat'), ('dog',), 1.0, 0.125),
 (('a', 'dog'), ('cat',), 1.0, 0.25)]

## FP Growth

We don't need to exclusively write our own functions for frequent itemset mining, however. PySpark has a built-in parallel implementation of FP Growth -- a highly scalable approach to mining frequent itemsets and association rules:

In [2]:
from pyspark.ml.fpm import FPGrowth

fp = FPGrowth()
fpm = fp.fit(df.select(df.basket.alias('items')))

fpm.freqItemsets.sort("freq", ascending=False).show()

+---------------+----+
|          items|freq|
+---------------+----+
|          [dog]|   7|
|          [cat]|   6|
|          [and]|   5|
|     [cat, dog]|   5|
|     [and, cat]|   4|
|     [and, dog]|   4|
|     [training]|   3|
|[and, cat, dog]|   3|
|            [a]|   3|
|  [a, cat, dog]|   3|
|       [a, cat]|   3|
|       [a, dog]|   3|
+---------------+----+



In [3]:
fpm.associationRules.sort("antecedent", "consequent").show()

+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
|       [a]|     [cat]|               1.0|1.3333333333333333|  0.375|
|       [a]|     [dog]|               1.0|1.1428571428571428|  0.375|
|  [a, cat]|     [dog]|               1.0|1.1428571428571428|  0.375|
|  [a, dog]|     [cat]|               1.0|1.3333333333333333|  0.375|
|     [and]|     [cat]|               0.8|1.0666666666666667|    0.5|
|     [and]|     [dog]|               0.8|0.9142857142857144|    0.5|
|     [cat]|     [dog]|0.8333333333333334|0.9523809523809524|  0.625|
+----------+----------+------------------+------------------+-------+



It is built to scale for large datasets -- e.g. [Instacart's data of over 3 million orders on the platform](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) -- which can allow us to discover patterns and association rules in food buying that may point to deeper cultural forces at play.

In [5]:
instacart_orders = spark.read.csv('/project/macs40123/order_products_prior.csv', header=True)
instacart_baskets = instacart_orders.groupBy('order_id').agg(F.collect_list('product_id').alias('basket'))
# for demo with 8 cores, 5 GB RAM each, sample down:
instacart_baskets = instacart_baskets.sample(0.01, seed=40123)

In [6]:
fp = FPGrowth(minConfidence=0.5, minSupport=0.001) # support = 3.21m * 0.001 = 321
fpm = fp.fit(instacart_baskets.select(instacart_baskets.basket.alias('items')))
fpm.freqItemsets.sort("freq", ascending=False).show(5)

+-------+----+
|  items|freq|
+-------+----+
|[24852]|4723|
|[13176]|3793|
|[21137]|2593|
|[21903]|2418|
|[47209]|2135|
+-------+----+
only showing top 5 rows



In [7]:
# lift > 1 (pos corr), lift < 1 (neg corr), lift == 1 (indep)
fpm.associationRules.sort("antecedent", "consequent").show(truncate=False)

+---------------------+----------+------------------+------------------+---------------------+
|antecedent           |consequent|confidence        |lift              |support              |
+---------------------+----------+------------------+------------------+---------------------+
|[21709, 35221]       |[44632]   |0.524390243902439 |24.01497860199715 |0.0013394386817431392|
|[21709, 44632]       |[35221]   |0.5119047619047619|37.519814090019565|0.0013394386817431392|
|[25890, 49683]       |[24852]   |0.532258064516129 |3.6178447269026655|0.0010279413138958977|
|[27966, 47209, 21137]|[13176]   |0.5176470588235295|4.381234782338984 |0.0013705884185278635|
|[28204, 16797]       |[24852]   |0.5735294117647058|3.898372793961963 |0.0012148397346042426|
|[33787, 18523]       |[33754]   |0.6923076923076923|73.83772041911577 |0.00112139052425007  |
|[38544]              |[4962]    |0.5138888888888888|222.93749999999997|0.0011525402610347943|
|[41065]              |[45007]   |0.52845528455284

In [20]:
product_codes = spark.read.csv('/project/macs40123/products.csv', header=True)

# discovery of a recipe? -- a particular type of cultural assemblage
product_codes.select('product_id', 'product_name') \
             .where(F.col('product_id').isin([25890, 49683, 24852])) \
             .show(truncate=False)

# grammar of *organic* fruit/vegetables: Avacado + Raspberries + Strawberries -> Bananas
product_codes.select('product_id', 'product_name') \
             .where(F.col('product_id').isin([27966, 47209, 21137, 13176
                                              ])) \
             .show(truncate=False)

# indication of toddler food? "Yotoddler"
product_codes.select('product_id', 'product_name') \
             .where(F.col('product_id').isin([4962, 38544
                                              ])) \
             .show(truncate=False)

+----------+---------------------------------+
|product_id|product_name                     |
+----------+---------------------------------+
|24852     |Banana                           |
|25890     |Boneless Skinless Chicken Breasts|
|49683     |Cucumber Kirby                   |
+----------+---------------------------------+

+----------+----------------------+
|product_id|product_name          |
+----------+----------------------+
|13176     |Bag of Organic Bananas|
|21137     |Organic Strawberries  |
|27966     |Organic Raspberries   |
|47209     |Organic Hass Avocado  |
+----------+----------------------+

+----------+-----------------------------------------------------+
|product_id|product_name                                         |
+----------+-----------------------------------------------------+
|4962      |Yotoddler Organic Pear Spinach Mango Yogurt          |
|38544     |Organic Whole Milk Strawberry Beet Berry Yogurt Pouch|
+----------+----------------------------------